# 📦 Формат: бинарный .bin (word2vec Binary Format)

Это не обычный текстовый файл, а бинарная модель, созданная с помощью Word2Vec (CBOW или Skip-gram).

Размер векторов: 300

Используется с библиотекой gensim:


### 🔧 Пример загрузки в Python:



In [ ]:
from gensim.models import KeyedVectors

# Загрузка бинарного файла
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# Пример использования
print(model['cat'])           # вектор (300 чисел)
print(model.most_similar('king'))  # похожие слова


### ✅ Варианты получить word2vec для узбекского:


1. FastText для узбекского (можно конвертировать в .bin)


In [ ]:
import requests
import gzip
import shutil

# 1. Скачиваем файл .gz
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uz.300.vec.gz"
gz_filename = "cc.uz.300.vec.gz"
vec_filename = "cc.uz.300.vec"

print("Скачиваем файл...")
response = requests.get(url, stream=True)
with open(gz_filename, 'wb') as f:
    shutil.copyfileobj(response.raw, f)
print("Файл скачан:", gz_filename)

# 2. Распаковываем .gz
print("Распаковываем файл...")
with gzip.open(gz_filename, 'rb') as f_in:
    with open(vec_filename, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
print("Файл распакован:", vec_filename)


SyntaxError: invalid syntax (4135119900.py, line 1)

In [ ]:
from gensim.models import KeyedVectors

# Загрузка текстовой модели
model = KeyedVectors.load_word2vec_format('cc.uz.300.vec', binary=False)

# Сохранение в бинарный .bin формат
model.save_word2vec_format('uzbek-word2vec.bin', binary=True)


### Получить файл uzbek_embedding.txt (аналог yelp_embedding.txt) в формате:

## План действий:
# Шаг 1. 📦 Найти или собрать узбекский корпус отзывов
Варианты:
Uzbek Wikipedia (нейтральный, не отзывы, но пригодится)

Платформы типа olx.uz, booking.com, telegram отзыв-боты (можно парсить)

Я могу автоматом собрать мелкий набор отзывов как пример — подходит?

# Шаг 2. 🤖 Обучить модель Word2Vec
Используем gensim

Можно задать:

размер векторов (например, 100)

min_count (сколько раз слово должно встретиться)

# Шаг 3. 💾 Сохранить в uzbek_embedding.txt
Сохраняем в .txt формате: слово + 100 векторов


📥 Шаг 1: Загрузка и подготовка данных


In [3]:
import requests

url = "https://huggingface.co/datasets/Sanatbek/Uzbek-restaurant-domain-sentiment-reviews/resolve/main/uzbek_restaurant_supervised_reviews.csv"

response = requests.get(url)

with open("uz_reviews.csv", "wb") as f:
    f.write(response.content)

print("Файл успешно загружен как uz_reviews.csv")


Файл успешно загружен как uz_reviews.csv


In [5]:
import pandas as pd

# Загрузка CSV-файла
df = pd.read_csv('uz_reviews.csv')

# Сохраняем только колонку "review" в текстовый файл без заголовков и индексов
df['review'].to_csv('uz_reviews.txt', index=False, header=False)


🧠 Шаг 2: Обучение модели Word2Vec


In [7]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Загрузка и токенизация отзывов
with open('uz_reviews.txt', 'r', encoding='utf-8') as f:
    corpus = f.readlines()

sentences = [simple_preprocess(line) for line in corpus]

# Обучение модели Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

# Сохранение модели в формате .txt
model.wv.save_word2vec_format('uzbek_embedding.txt', binary=False)


Как получить эмбеддинги для узбекского языка в формате yelp_words.txt?
1. Найти корпус отзывов на узбекском
- Например, ты уже упоминал датасет:
https://huggingface.co/datasets/Sanatbek/Uzbek-restaurant-domain-sentiment-reviews
Это именно отзывы на узбекском, можно использовать их.

2. Подготовить корпус текстов
- Извлечь все отзывы из датасета, сохранить в файл uzbek_reviews.txt (каждый отзыв — отдельная строка)

3. Обучить эмбеддинги Word2Vec (или FastText) на этом корпусе

In [26]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

with open('uz_reviews.txt', 'r', encoding='utf-8') as f:
    corpus = f.readlines()

sentences = [simple_preprocess(line) for line in corpus]

model = Word2Vec(sentences, vector_size=300, window=5, min_count=2, workers=4)

model.wv.save_word2vec_format('yelp_words.txt', binary=False)


4. В итоге получишь файл yelp_words.txt — эмбеддинги узбекских слов из отзывов.

Полный пример скачивания датасета, подготовки и обучения:



In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import requests

# 1. Скачиваем датасет (если еще не скачан)
url = "https://huggingface.co/datasets/Sanatbek/Uzbek-restaurant-domain-sentiment-reviews/resolve/main/uzbek_restaurant_supervised_reviews.csv"
csv_file = "uzbek_reviews.csv"
r = requests.get(url)
with open(csv_file, 'wb') as f:
    f.write(r.content)

# 2. Загружаем отзывы
df = pd.read_csv(csv_file)
reviews = df['review'].dropna().tolist()

# 3. Сохраняем отзывы в текстовый файл (каждый отзыв на отдельной строке)
with open('uz_reviews.txt', 'w', encoding='utf-8') as f:
    for review in reviews:
        if isinstance(review, str) and review.strip():
            f.write(review.strip() + '\n')

# 4. Загружаем и токенизируем
with open('uz_reviews.txt', 'r', encoding='utf-8') as f:
    corpus = f.readlines()
sentences = [simple_preprocess(line) for line in corpus]

# 5. Обучаем Word2Vec
model = Word2Vec(sentences, vector_size=300, window=5, min_count=2, workers=4)

# 6. Сохраняем в текстовом формате (как yelp_words.txt)
model.wv.save_word2vec_format('yelp_words.txt', binary=False)


### ✅ Как обучить GloVe на узбекском (на своём корпусе)


✅ Установи нужные библиотеки:



In [13]:
pip install pandas requests glove-python-binary gensim


1: Использовать альтернативу GloVe — FastText


In [17]:
import requests
import gzip
import shutil

# Скачиваем архив с векторами FastText
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uz.300.vec.gz"
gz_file = "cc.uz.300.vec.gz"
vec_file = "cc.uz.300.vec"

# Скачиваем
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(gz_file, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

# Распаковываем
with gzip.open(gz_file, 'rb') as f_in:
    with open(vec_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(f"Векторный файл {vec_file} готов к использованию.")


Векторный файл cc.uz.300.vec готов к использованию.


1. Загрузить векторы с помощью gensim



In [19]:
from gensim.models import KeyedVectors

# Загружаем векторы (не бинарный формат)
model = KeyedVectors.load_word2vec_format('cc.uz.300.vec', binary=False)

# Проверим похожие слова для примера
print(model.most_similar('yaxshi'))  # "хороший" по-узбекски


[('yaxshi-yaxshi', 0.6805984973907471), ('-yaxshi', 0.6398767828941345), ('Engyaxshi', 0.6159312129020691), ('yomon-yaxshi', 0.6139829158782959), ('374yaxshi', 0.6131339073181152), ('Yaxshi-yaxshi', 0.6108852624893188), ('yaxshii', 0.5992532968521118), ('yaxshi-yu', 0.5914546251296997), ('axshi', 0.5860336422920227), ('yaxshi-yomon', 0.5804401636123657)]


2. Использовать векторы для задач NLP
    - Поиск синонимов/похожих слов

    - Кластеризация слов

    - Векторизация отзывов (усреднение эмбеддингов слов)

    - Использование как эмбеддинги в классификаторах (например, для анализа тональности)

3. Пример: как получить вектор слова



In [20]:
vector = model['oshxona']  # 'кухня' по-узбекски
print(vector[:10])  # первые 10 элементов вектора


[ 0.0088  0.044  -0.022   0.0036 -0.0078  0.0118 -0.0605 -0.1067 -0.0035
  0.0115]


🐍 Полный Python-скрипт:



Обучить Word2Vec на твоём корпусе — простой пример


In [22]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import pandas as pd
import requests
import os

# Скачиваем датасет, если нет
url = "https://huggingface.co/datasets/Sanatbek/Uzbek-restaurant-domain-sentiment-reviews/resolve/main/uzbek_restaurant_supervised_reviews.csv"
csv_filename = "uz_reviews.csv"
if not os.path.exists(csv_filename):
    r = requests.get(url)
    with open(csv_filename, 'wb') as f:
        f.write(r.content)

# Читаем отзывы и токенизируем
df = pd.read_csv(csv_filename)
reviews = df['review'].dropna().tolist()
sentences = [simple_preprocess(text) for text in reviews if text.strip() != ""]

# Обучаем Word2Vec
model = Word2Vec(sentences, vector_size=300, window=5, min_count=2, workers=4)

# Сохраняем в формате текста (аналог GloVe)
model.wv.save_word2vec_format('uzbek_word2vec_300d.txt', binary=False)

print("✅ Модель Word2Vec обучена и сохранена в uzbek_word2vec_300d.txt")


✅ Модель Word2Vec обучена и сохранена в uzbek_word2vec_300d.txt


### Word2Vec (uzbek_word2vec_300d.txt) получить файл в формате GloVe (glove_words.txt), то есть:

## Задача: преобразовать uzbek_word2vec_300d.txt в glove_words.txt
Часто в word2vec-файлах есть первая строка — заголовок (кол-во слов и размерность), а в GloVe его нет.

Поэтому:

- Нужно прочитать uzbek_word2vec_300d.txt

- Пропустить первую строку (если там заголовок)

- Записать остальные строки в новый файл без изменений



In [24]:
input_file = 'uzbek_word2vec_300d.txt'
output_file = 'uzbek_glove_words.txt'

with open(input_file, 'r', encoding='utf-8') as fin, open(output_file, 'w', encoding='utf-8') as fout:
    first_line = fin.readline()
    # Проверяем, есть ли заголовок (если первая строка содержит два числа)
    if len(first_line.strip().split()) == 2:
        print(f"Пропускаем заголовок: {first_line.strip()}")
    else:
        # Если заголовка нет, то пишем первую строку
        fout.write(first_line)
    
    # Копируем оставшиеся строки
    for line in fin:
        fout.write(line)

print(f"Файл {output_file} создан в формате GloVe.")


Пропускаем заголовок: 3915 300
Файл uzbek_glove_words.txt создан в формате GloVe.
